# 🏆 K-League 패스 예측 - 완전 자동화 노트북

> **한 번 설정하면 끝! 이후에는 "Run All"만!**
> 
> **작성:** 2025-12-15
> **현재:** CV 16.04 (Zone 6x6: 16.36)
> **목표:** Public < 16.0 (상위 20%)

---

## 📁 첫 실행 시에만! (이후 자동)

**Google Drive 폴더 구조:**
```
MyDrive/kleague-algorithm/
└── data/
    ├── train.csv          ← 여기에 업로드! (한 번만)
    ├── test.csv           ← 여기에 업로드!
    ├── sample_submission.csv
    └── test/              ← 폴더 전체 업로드!
```

**이후에는 자동!** 🎉

---
# 🔧 자동 설정 (실행만 하면 됨)

In [ ]:
# ============================================================================
# 1. 환경 설정 (자동)
# ============================================================================
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=False)  # 이미 마운트되면 스킵

# 프로젝트 경로 설정
PROJECT_ROOT = Path('/content/drive/MyDrive/kleague-algorithm')
DATA_DIR = PROJECT_ROOT / 'data'
MODEL_DIR = PROJECT_ROOT / 'models'
SUBMIT_DIR = PROJECT_ROOT / 'submissions'
LOG_DIR = PROJECT_ROOT / 'logs'

# 폴더 자동 생성
for dir_path in [PROJECT_ROOT, DATA_DIR, MODEL_DIR, SUBMIT_DIR, LOG_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

os.chdir(PROJECT_ROOT)

print("✅ 환경 설정 완료!")
print(f"   Working dir: {os.getcwd()}")
print(f"   Data dir: {DATA_DIR}")

In [ ]:
# ============================================================================
# 2. 데이터 확인 (자동)
# ============================================================================
import glob

required_files = ['train.csv', 'test.csv', 'sample_submission.csv']
missing_files = []

for file in required_files:
    file_path = DATA_DIR / file
    if file_path.exists():
        print(f"✅ {file}: {file_path.stat().st_size / 1024 / 1024:.1f} MB")
    else:
        print(f"❌ {file}: NOT FOUND")
        missing_files.append(file)

# test/ 폴더 확인
test_folder = DATA_DIR / 'test'
if test_folder.exists():
    test_files = list(test_folder.glob('*.csv'))
    print(f"✅ test/: {len(test_files)} files")
else:
    print(f"❌ test/ folder: NOT FOUND")
    missing_files.append('test/')

if missing_files:
    print("\n" + "="*80)
    print("⚠️ 데이터 업로드 필요!")
    print("="*80)
    print(f"\n누락: {missing_files}")
    print(f"\n업로드 위치: {DATA_DIR}")
    print("\n1. 왼쪽 Files 탭 클릭")
    print("2. drive/MyDrive/kleague-algorithm/data/ 이동")
    print("3. 파일 업로드 (우클릭 → Upload)")
    print("4. 이 셀 다시 실행")
    raise FileNotFoundError(f"데이터 업로드 필요: {missing_files}")
else:
    print("\n✅ 모든 데이터 준비 완료!")

In [ ]:
# ============================================================================
# 3. GPU 확인 & 라이브러리 설치 (자동)
# ============================================================================
import torch

print("GPU 확인...")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    USE_GPU = True
else:
    print("⚠️ CPU 모드 (런타임 → GPU 변경 권장)")
    USE_GPU = False

# 라이브러리 설치 (없으면 자동 설치)
print("\n라이브러리 설치...")
!pip install -q catboost
print("✅ CatBoost 설치 완료!")

---
# 🤖 모델 코드 (자동)

In [ ]:
# ============================================================================
# 4. FastExperiment 클래스
# ============================================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import time

class FastExperiment:
    """빠른 실험 시스템 (Colab 버전)"""
    
    def __init__(self, n_folds=5, random_state=42):
        self.n_folds = n_folds
        self.random_state = random_state
        np.random.seed(random_state)
    
    def load_data(self, train_path):
        """데이터 로드"""
        print(f"\n{'='*80}")
        print("데이터 로드")
        print(f"{'='*80}")
        
        train_df = pd.read_csv(train_path)
        print(f"  Train: {len(train_df):,}개 패스")
        print(f"  Episodes: {train_df['game_episode'].nunique():,}개")
        
        return train_df
    
    def create_features(self, df):
        """피처 생성 (Episode 독립성 유지!)"""
        df = df.copy()
        
        # Zone 6x6
        df['zone_x'] = (df['start_x'] / (105/6)).astype(int).clip(0, 5)
        df['zone_y'] = (df['start_y'] / (68/6)).astype(int).clip(0, 5)
        
        # Direction 8-way
        df['dx'] = df['end_x'] - df['start_x']
        df['dy'] = df['end_y'] - df['start_y']
        df['prev_dx'] = df.groupby('game_episode')['dx'].shift(1).fillna(0)
        df['prev_dy'] = df.groupby('game_episode')['dy'].shift(1).fillna(0)
        
        angle = np.degrees(np.arctan2(df['prev_dy'], df['prev_dx']))
        df['direction'] = ((angle + 22.5) // 45).astype(int) % 8
        
        # Goal features
        df['goal_distance'] = np.sqrt((105 - df['start_x'])**2 + (34 - df['start_y'])**2)
        df['goal_angle'] = np.degrees(np.arctan2(34 - df['start_y'], 105 - df['start_x']))
        
        # Time features
        df['time_left'] = 5400 - df['time_seconds']
        
        # Episode features
        df['pass_count'] = df.groupby('game_episode').cumcount() + 1
        
        print(f"  피처 생성 완료")
        return df
    
    def prepare_data(self, df):
        """마지막 패스 추출 & Feature/Target 분리"""
        train_last = df.groupby('game_episode').last().reset_index()
        
        self.feature_cols = [
            'start_x', 'start_y',
            'zone_x', 'zone_y',
            'direction',
            'goal_distance', 'goal_angle',
            'period_id', 'time_seconds', 'time_left',
            'pass_count',
            'prev_dx', 'prev_dy'
        ]
        
        X = train_last[self.feature_cols].values
        y = train_last[['end_x', 'end_y']].values
        groups = train_last['game_episode'].str.split('_').str[0].values
        
        print(f"\n  Episodes: {len(train_last):,}개")
        print(f"  Features: {len(self.feature_cols)}개")
        
        return X, y, groups

print("✅ FastExperiment 클래스 로드!")

---
# 🚀 실행 (자동)

**이제부터는 완전 자동!**

In [ ]:
# ============================================================================
# 5. 데이터 로드 & 피처 생성
# ============================================================================
exp = FastExperiment(n_folds=5, random_state=42)

# Train
train_df = exp.load_data(DATA_DIR / 'train.csv')
train_df = exp.create_features(train_df)
X, y, groups = exp.prepare_data(train_df)

print("✅ 데이터 준비 완료!")

In [ ]:
# ============================================================================
# 6. CatBoost 학습 (5-fold CV)
# ============================================================================
import catboost as cb

print("="*80)
print("CatBoost 학습 (Full Data, 5-fold)")
print("="*80)
print("\n목표: CV 16.04 (Zone 6x6: 16.36)")

cat_params = {
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'random_state': 42,
    'verbose': 0,
    'task_type': 'GPU' if USE_GPU else 'CPU'
}

gkf = GroupKFold(n_splits=5)
fold_scores = []

print(f"\nGPU: {USE_GPU}")
print(f"Training...")

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
    fold_start = time.time()
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Separate models for x and y
    model_x = cb.CatBoostRegressor(**cat_params)
    model_y = cb.CatBoostRegressor(**cat_params)
    
    model_x.fit(X_train, y_train[:, 0])
    model_y.fit(X_train, y_train[:, 1])
    
    pred_x = np.clip(model_x.predict(X_val), 0, 105)
    pred_y = np.clip(model_y.predict(X_val), 0, 68)
    
    dist = np.sqrt((pred_x - y_val[:, 0])**2 + (pred_y - y_val[:, 1])**2)
    cv = dist.mean()
    fold_scores.append(cv)
    
    fold_time = time.time() - fold_start
    print(f"  Fold {fold+1}: {cv:.4f}  ({fold_time:.1f}s)")

cv_mean = np.mean(fold_scores)
cv_std = np.std(fold_scores)

print(f"\n{'='*80}")
print(f"CV: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"Zone 6x6 Baseline: 16.36")
print(f"개선: {cv_mean - 16.36:+.2f}")
print(f"{'='*80}")

In [ ]:
# ============================================================================
# 7. Test 데이터 로드 & 피처 생성
# ============================================================================
print("\n" + "="*80)
print("Test 데이터 처리")
print("="*80)

test_meta = pd.read_csv(DATA_DIR / 'test.csv')
sample_sub = pd.read_csv(DATA_DIR / 'sample_submission.csv')

print(f"\n  Test episodes: {len(test_meta):,}개")

# Test episodes 로드
test_episodes = []
for _, row in test_meta.iterrows():
    ep_path = DATA_DIR / row['path']
    ep_df = pd.read_csv(ep_path)
    ep_df['game_episode'] = row['game_episode']
    test_episodes.append(ep_df)

test_df = pd.concat(test_episodes, ignore_index=True)
print(f"  Test 패스: {len(test_df):,}개")

# Features
test_df = exp.create_features(test_df)
test_last = test_df.groupby('game_episode').last().reset_index()
X_test = test_last[exp.feature_cols].values

print(f"\n✅ Test 데이터 준비 완료!")
print(f"   Test episodes: {len(test_last):,}개")

In [ ]:
# ============================================================================
# 8. Full Train 재학습 & Test 예측
# ============================================================================
print("\n" + "="*80)
print("Full Train 재학습 & Test 예측")
print("="*80)

# Full train으로 재학습
print("\nFull train 학습...")
model_x_final = cb.CatBoostRegressor(**cat_params)
model_y_final = cb.CatBoostRegressor(**cat_params)

model_x_final.fit(X, y[:, 0])
model_y_final.fit(X, y[:, 1])

print("✅ 학습 완료!")

# Predict
print("\nTest 예측...")
pred_x = np.clip(model_x_final.predict(X_test), 0, 105)
pred_y = np.clip(model_y_final.predict(X_test), 0, 68)

print("✅ 예측 완료!")

# 모델 저장 (선택)
model_x_final.save_model(str(MODEL_DIR / 'catboost_x.cbm'))
model_y_final.save_model(str(MODEL_DIR / 'catboost_y.cbm'))
print(f"\n✅ 모델 저장: {MODEL_DIR}")

In [ ]:
# ============================================================================
# 9. Submission 생성
# ============================================================================
print("\n" + "="*80)
print("Submission 생성")
print("="*80)

# Submission DataFrame
submission = pd.DataFrame({
    'game_episode': test_last['game_episode'],
    'end_x': pred_x,
    'end_y': pred_y
})

# Merge with sample_submission
submission = sample_sub[['game_episode']].merge(submission, on='game_episode', how='left')

# 검증
print(f"\n검증:")
print(f"  샘플 수: {len(submission)} (2414 기대)")
print(f"  NaN: {submission.isna().sum().sum()} (0 기대)")
print(f"  end_x 범위: [{submission['end_x'].min():.2f}, {submission['end_x'].max():.2f}]")
print(f"  end_y 범위: [{submission['end_y'].min():.2f}, {submission['end_y'].max():.2f}]")

assert len(submission) == 2414, "샘플 수 오류!"
assert submission.isna().sum().sum() == 0, "NaN 존재!"
assert submission['end_x'].min() >= 0 and submission['end_x'].max() <= 105, "end_x 범위 오류!"
assert submission['end_y'].min() >= 0 and submission['end_y'].max() <= 68, "end_y 범위 오류!"

# 저장
import datetime
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
filename = f'submission_catboost_cv{cv_mean:.2f}_{timestamp}.csv'
submission_path = SUBMIT_DIR / filename
submission.to_csv(submission_path, index=False)

print(f"\n✅ Submission 저장: {submission_path}")
print(f"\n샘플:")
print(submission.head(10))

In [ ]:
# ============================================================================
# 10. 자동 다운로드 (선택)
# ============================================================================
from google.colab import files

print("\n" + "="*80)
print("다운로드")
print("="*80)

# 다운로드 여부 선택 (True/False)
AUTO_DOWNLOAD = True  # 자동 다운로드

if AUTO_DOWNLOAD:
    print(f"\n📥 다운로드 시작: {filename}")
    files.download(str(submission_path))
    print("✅ 다운로드 완료!")
else:
    print(f"\n⏸️ 자동 다운로드 꺼짐")
    print(f"   수동 다운로드: 왼쪽 Files → {submission_path}")

print(f"\n💾 Google Drive 저장 위치:")
print(f"   {submission_path}")

---
# 🎯 최종 요약

In [ ]:
print("="*80)
print("최종 요약")
print("="*80)

print(f"\n[성능]")
print(f"  CV: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"  Zone 6x6: 16.36")
print(f"  개선: {cv_mean - 16.36:+.2f}")

print(f"\n[Fold별]")
for i, score in enumerate(fold_scores):
    print(f"  Fold {i+1}: {score:.4f}")

print(f"\n[예상]")
print(f"  예상 Public (Gap +1.0): {cv_mean + 1.0:.2f}")
print(f"  예상 Public (Gap +0.5): {cv_mean + 0.5:.2f}")
print(f"  Zone 6x6 Gap: +0.02")

print(f"\n[파일]")
print(f"  Submission: {filename}")
print(f"  위치: {SUBMIT_DIR}")
print(f"  Models: {MODEL_DIR}")

print(f"\n[다음 단계]")
print(f"  1. DACON에 {filename} 제출")
print(f"  2. Public Score 확인")
print(f"  3. Gap 분석")
print(f"     - Gap < 1.0 → Feature Engineering")
print(f"     - Gap > 2.0 → Regularization 강화")

print(f"\n{'='*80}")
print("✅ 완료! DACON 제출하세요!")
print(f"{'='*80}")

---
# 📝 설정 변경

## 자동 다운로드 On/Off

위의 **10. 자동 다운로드** 셀에서:
```python
AUTO_DOWNLOAD = True   # 자동 다운로드
AUTO_DOWNLOAD = False  # 수동 (Google Drive에만 저장)
```

## GPU On/Off

런타임 → 런타임 유형 변경 → GPU/CPU 선택

---

**"Run All" 한 번이면 끝!** 🎉